# 01. Import & Install Dependencies

In [1]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [97]:
import cv2                                # Computer Vison Library OpenCV
import numpy as np                        # numpy arrays structure our datasets (numpy instead of tensorflow)
import os                                 # work with operating systems file path
from matplotlib import pyplot as plt      # matplotlib just helps us visualize images a little bit easier
import mediapipe as mp                    # mediapipe (https://google.github.io/mediapipe/)
from math import *

# 02. Using Mediapipe (Google's open-source framework)

In [98]:
mp_face_mesh = mp.solutions.face_mesh              # FaceMesh Model has -- 478 -- Keypoints 
mp_drawing_styles = mp.solutions.drawing_styles    # Drawing styles
mp_drawing = mp.solutions.drawing_utils            # Drawing utilites

## Given the original image and obtaining the values through the model.

In [99]:
def mediapipe_detection(image,model):                 
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)   # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                     # Image is no longer writeable
    results = model.process(image)                    # Make prediction
    image.flags.writeable = True                      # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)    # COLOR CONVERSION RGB 2 BGR
    return image, results

## Calculating blink rate (Euclidean distance).

In [100]:
# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)

    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio

# Euclaidean distance
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = sqrt((x1 - x)**2 + (y1 - y)**2)
    if 1> distance:
        return 1
    else:
        return distance

## Storing the keypoints values in the mediapipe into a variable.

In [101]:
# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]
 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

# 03. Extract Position Landmarks (Keypoints Values) 

In [103]:
def extract_landmarks(img, results):
    frame = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    mesh_coords = mesh_coord_detection(img, results)
    
    # extract face 478 keyponits
    face = np.array([[res.x, res.y, res.z] for res in results.multi_face_landmarks[0].landmark]).flatten() if results.multi_face_landmarks[0] else np.zeros(1404)
    
    # extract right-eye 16 keypoints
    right_eye_coords = np.array([mesh_coords[p] for p in RIGHT_EYE]).flatten()  
    
    # extract left-eye 16 keypoints
    left_eye_coords = np.array([mesh_coords[p] for p in LEFT_EYE]).flatten()    
    
    # calculating eye ratio
    ratio = np.array(blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)).flatten()  
    
    # return all feature values
    return np.concatenate([face,right_eye_coords,left_eye_coords,ratio])


def mesh_coord_detection(img, results):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]

    # returning the list of tuples for each landmarks 
    return mesh_coord

# 04. Setup Folder for Collection

In [104]:
DATA_PATH = os.path.join('ML_Data')         # Path for exported data, numpy arrays

actions = np.array(['true', 'lie'])       # Actions that we try to detect

no_sequences = 21                       # videos worth of data per action (True-02 vdos, Lie-02 Vdos)

sequence_length = 30              # Videos are going to be 30 frames in length

In [105]:
# Create Folders for save keypoints values

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

#  05. Collect Keypoints Values for Training and Testing

In [106]:
for action in actions:   # Loop through actions
    
    for sequence in range(no_sequences):          # Loop through sequences (videos)
        
        VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/true/0.mp4
        cap = cv2.VideoCapture(VDO_PATH)                # Read Video
        
        with mp_face_mesh.FaceMesh(                         # set mediapipe model
            max_num_faces =1,
            refine_landmarks = True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as face_mesh:
            for frame_num in range(sequence_length):

                ret, frame = cap.read()                                # Read feed

                print("Processing Video Path:",VDO_PATH)                      # Processing Video Path:Datasets/true/0/0.mp4

                image, results = mediapipe_detection(frame, face_mesh)                        # Make detection
    
                feature_values = extract_landmarks(image,results)                   # Extract face , left-eye , right-eye keypoints
            
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # create export keypoints local path
            
                print("feature_valuess Shape:", feature_values.shape)                       # Keypoints Shape:(1499,) ---> (1434 + 32 + 32 + 1)
                
                np.save(npy_path, feature_values)                                   # Save Keypoints values

            cap.release()     # Release feed
            
            cv2.waitKey(1)    # wait for 1 Secound
            
            print("____________Finish__________________")                  

Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/0/0.mp4
feature_valuess Shape: (1499,)
Processing V

Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/3/3.mp4
feature_valuess Shape: (1499,)
____________Finish__________________
Processing Video Path: Datasets/true/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/4/4.mp4
featur

Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/7/7.mp4
feature_valuess Shape: (1499,)
Processing V

feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/11/11.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/t

Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/true/14/14.mp4
feature_valuess S

...

Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/4/4.mp4
feature_valuess Shape: (1499,)
____________Finish__________________
Processing Video Path: Datasets/lie/5/5.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/5/5.mp4
feature_valuess Sha

Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/8/8.mp4
feature_valuess Shape: (1499,)
____________Finish_______

Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/12/12.mp4
feature_valuess Shape: (1499,)

Processing Video Path: Datasets/lie/15/15.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/15/15.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/15/15.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/15/15.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/15/15.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/15/15.mp4
feature_valuess Shape: (1499,)
____________Finish__________________
Processing Video Path: Datasets/lie/16/16.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/16/16.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/16/16.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/16/16.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/16/16.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/16/16.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/16/

Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)
Processing Video Path: Datasets/lie/19/19.mp4
feature_valuess Shape: (1499,)

# 6. Preprocess Data and Create Labels and Features

In [107]:
from sklearn.model_selection import train_test_split # Scikit-learn for our evaluation metrics
from tensorflow.keras.utils import to_categorical   # Converting Data (Encoded)

In [108]:
label_map = {label:num for num, label in enumerate(actions)}  # Actions Label Map  {'true': 0, 'lie': 1} 

In [109]:
sequences, labels = [], []                # like : Features(X) Data , labels(Y) Data
for action in actions:              
    for sequence in range(no_sequences):  
        values = []                       # Save all of diff.frames paricular sequence
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))  # load np array
            values.append(res)                # Append all res 
        sequences.append(values)             # Append all values
        labels.append(label_map[action])   # Append all actions on labels

In [110]:
X_train.shape

(39, 50, 1498)

In [121]:
X = np.array(sequences)  # sequences
y = to_categorical(labels).astype(int)  # labels

# Train & Testing Partition
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) 

In [112]:
# X.shape : (42, 30, 1499)  -----> ( Sequences(videos) , framers per vdo, keypoints shape [(478 * 3) + (32 * 2) + 1] )
# y.shape : (42, 2) -------->   ( Sequences(videos), labels[actions] )

# X_train.shape : (39, 30, 1499) 
# X_test.shape : (3, 30, 1499)

# y_train.shape : (39, 2) 
# y_test.shape : (3, 2) 

In [122]:
X.shape

(42, 30, 1499)

In [123]:
y.shape

(42, 2)

In [124]:
X_train.shape

(39, 30, 1499)

In [125]:
X_test.shape

(3, 30, 1499)

In [126]:
y_train.shape

(39, 2)

In [127]:
y_test.shape

(3, 2)

# 7. Build and Train LSTM Neural Network

In [128]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [129]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [130]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1499)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [131]:
actions.shape[0]

2

In [132]:
res = [.7, 0.2, 0.1]

In [133]:
actions[np.argmax(res)]

'true'

In [134]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [138]:
model.fit(X_train, y_train, epochs=10, callbacks=[tb_callback])

Epoch 1/10
2/2 [==============================] - 1s 844ms/step - loss: 375.0261 - categorical_accuracy: 0.4615
Epoch 2/10
2/2 [==============================] - 0s 59ms/step - loss: 187.0708 - categorical_accuracy: 0.5641
Epoch 3/10
2/2 [==============================] - 0s 61ms/step - loss: 199.3580 - categorical_accuracy: 0.4103
Epoch 4/10
2/2 [==============================] - 0s 60ms/step - loss: 172.4189 - categorical_accuracy: 0.5385
Epoch 5/10
2/2 [==============================] - 0s 59ms/step - loss: 134.9552 - categorical_accuracy: 0.4872
Epoch 6/10
2/2 [==============================] - 0s 66ms/step - loss: 192.3743 - categorical_accuracy: 0.4359
Epoch 7/10
2/2 [==============================] - 0s 59ms/step - loss: 152.3627 - categorical_accuracy: 0.4103
Epoch 8/10
2/2 [==============================] - 0s 64ms/step - loss: 112.5865 - categorical_accuracy: 0.6923
Epoch 9/10
2/2 [==============================] - 0s 59ms/step - loss: 147.9107 - categorical_accuracy: 0.5128


In [139]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 30, 64)            400384    
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 66        
Total params: 554,914
Trainable params: 554,914
Non-trainable params: 0
________________________________________________

# 8. Make Predictions

In [140]:
res = model.predict(X_test)

In [141]:
res

array([[0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [2.7822821e-06, 9.9999726e-01]], dtype=float32)

In [142]:
np.sum(res[0])

1.0

In [143]:
actions[np.argmax(res[0])]

'lie'

In [144]:
y_test

array([[0, 1],
       [1, 0],
       [1, 0]])

In [145]:
actions[np.argmax(y_test[0])]

'lie'

# 9. Save Weights

In [146]:
model.save('action.h5')

In [33]:
del model

In [147]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [148]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [149]:
yhat = model.predict(X_test)

In [150]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [161]:
plt.figure(figsize=(18,18))

<Figure size 1296x1296 with 0 Axes>

<Figure size 1296x1296 with 0 Axes>

In [160]:
plt.imshow(prob_viz(results, actions, image, colors))

TypeError: 'type' object is not iterable

In [151]:
ytrue

[1, 0, 0]

In [152]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 0],
        [2, 0]],

       [[0, 2],
        [0, 1]]], dtype=int64)

In [43]:
multilabel_confusion_matrix??

In [153]:
accuracy_score(ytrue, yhat)

0.3333333333333333

# 11. Test in Real Time

In [154]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [165]:
def draw_landmarks(image,face_landmarks):
    # Draw face connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec= None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
    
    # Draw face countours (FACEMESH_CONTOURS = frozenset().union(*[FACEMESH_LIPS, FACEMESH_LEFT_EYE, FACEMESH_LEFT_EYEBROW, FACEME) connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec= None,
            connection_drawing_spec= mp_drawing_styles.get_default_face_mesh_contours_style())
    
    # Draw eye iris ( FACEMESH_IRISES = frozenset().union(*[FACEMESH_LEFT_IRIS, FACEMESH_RIGHT_IRIS]) ) connections
    mp_drawing.draw_landmarks(
            image= image,
            landmark_list= face_landmarks,
            connections= mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec= None,
            connection_drawing_spec= mp_drawing_styles.get_default_face_mesh_iris_connections_style())

In [168]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, face_mesh)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_landmarks(image , results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Lie Detection', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


AttributeError: type object 'SolutionOutputs' has no attribute 'landmark'

In [169]:
cap.release()
cv2.destroyAllWindows()